# Taxi company car allocation


### Optimization Methods, MA06002, 2016

* Artem Korenev
* Andrei Kvasov
* Anton Marin
* Oleg Sudakov

It is common knowledge, that the financial efficiency, or, equivalently, profit of a taxi company depends on how many customers its drivers will be able to serve, and what resources should be spent to achieve such result. The company should also consider its reputation as a brand, which depends on, among other factors, on how fast its cars are able to pick up clients and to drive them to their destination. 

Both of these metrics greatly depend on how many cars a company should employ, and where they should be allocated geographically during the day.

In this problem, our team will examine historical data of Uber pickups in New York City, collected from April to September 2014.

# Problem formulation

The dataset contains the following information about Uber pickups in New York City: Date and time of the Uber pickup, the latitude of the Uber pickup, and the longitude of it. The pickup locations will be represented as points of a graph, where the weight of a given edge between vertices will be the distance between two points. We will be solving vehicle allocation problem, assuming that we know the orders' locations and times in advance.

Let's examine elements of the problem individually:

## Taxi drivers

The total number of taxi drivers will be denoted as $N$. Each driver will start his working day at position $X_0^j$ at time $T_0^j$, where $j$ denotes the index of a taxi car.

We will assume, that the driver's shift has a following schedule:

1. The car $j$ waits for orders at a specific point $X_w^j$, starting from the time $T_w^j$;
2. The car drives to pick up the client $k$ at the same point at time $T_p^k$;
3. After picking up the client, car immediately drives to its destination $X_d$. It arrives at time $T_a^k$;
4. The car drops the client at the destination point and afterwards heads to the next pickup point $X_{w+1}^j$ to wait for the order. It starts waiting for the next client at time $T_{w+1}^j$ (if the car waits for the next client at the drop-off point of the previous order, $T_{w+1}^j = T_a^k$.

Obviously, $T_a^k = T_p^k + time(X_p, X_d)$, and $T_{w+1}^j = T_a^k + time(X_d, X_{w+1}^j)$, where the $time(A, B)$ denotes the time needed to get from point $A$ to point $B$. $T_p^k$ is given by the dataset.

We will introduce two penalties for each car:

* $c_{d}$ - downtime cost per unit of time, induced on each taxi car when it is stationary.
* $c_{f}$ - fuel cost per unit of time, induced on each taxi car when it is en route.

Therefore, if the car $j$ is assigned to order $k$, the penalty for this individual order equals to:

$$P_{j, k} = time(X_w^j, X_p)c_{f} + (T_p^k - T_w^j - time(X_w^j, X_p))c_{d} + time(X_p, X_d)c_{f} + time(X_d, X_{w+1}^j)c_{f}$$

This formula does not take into account the downtime at the end of the day. Given end of workshift time $T_{end}$, car index $j$ and the number of orders that this car completed during the day as $n_j$ the penalty for the downtime at the end of the day for this car is $$E_j = (T_{end} - T^j_{n_j})c_{d}$$

The penalty for maintaining individual car will be denoted as $c_{a}$ ($a$ stands for auto). Then, the additional penalty for maintaining $N$ cars on a given day equals to $Nc_a$

## Clients

The total number of clients, or orders will be denoted as $C$. Individual order is represented by a tuple $(t^k, X_p, X_d)$, where $t^k$ denotes the minimum possible time for a pickup, $X_p$ the pickup point, and $X_d$ the destination point. 

A cost for a late pickup of the client will be denoted as $c_{t}$. Therefore, for individual order $k$ the penalty for the pickup delay equals to $D_k = (T_p^k - t^k)c_{t}$.

## Total penalty

To sum up, we will aim to minimize taxi car downtime, pickup delays and fuel costs. Thus, for a given formulation of a problem our goal is:

$$\Sigma_{j, k = 1}^{N, C}P_{j, k} + \Sigma_{j=1}^N E_j + Nc_a + \Sigma_{k=1}^C D_k \rightarrow min$$

# Chosen Methods

## 1. Greedy Algorithms

One of the most straightforward approaches to the problem are greedy algorithms. We can perform the greedy steps in order to make decisions upon pick ups of clients. However, it can be clearly seen almost instantly that applying such algorithms will not produce an optimal solution. However, applying such technique we can reach a sufficient approximation.

### 1.1 Straightforward greedy algorithm

For a greedy algorithm we choose the following strategy:

1. Sort the clients by their pickup time.
2. Iterate through a clients. For a given client $i$ assign the driver to $k$ to this client with the minimal possible value of a function: $time(X_k, X_p^i) c^f + (T_p^i - T_k - time(X_k, X_p^i)) c^d + (-T_p^i + T_k + time(X_k, X_p^i)) c_t$. Where $T_k$ is the time for $k$th driver after delivering its previous client and $X_k$ is the position where the driver $k$ delivered his or her last client. After choosing the best possible driver we assign the client to him or her and update the values $T_k$ and $X_k$ according to the data from the client's order.
3. We repeat the 2nd step until all clients are assigned to all drivers.

### 1.2 Simulated annealing algorithm

Since our previous algorithm has an obvious issue of falling into local optima, one possible improvement can be to perform simmulated annealing technique.

Consider that our initial solution is a random assignment of all clients to different clients and also in random order. To cover the full space of the solutions we can perform two different optimization steps:
1. Take a client $i$ assigned and assign it to a random driver.
2. Take a driver $k$, find two clients that are assigned to this driver. Swap the order of the delivering these clients.

## 2. Dynamic Programming

The second approach to the car allocation problem solution would be to use dynamic programming. Then, it could be solved using the same method, as speed scheduling problem, which was considered in the course. Given some optimal car assignment to orders up to the order $j$ (if the orders are sorted in time), we find the optimal value for each driver $i$ if we decide to assign him or her to order $j+1$. 

Given the information about the previous orders, the penalty for current assignment can easily be calculated, as we have information about geographical location of previous drop-off point, drop-off time, time and distance between previous drop-off point and current pickup point.

To solve such problem, a matrix of size $N \times C$ will be used, where $C$ denotes the number of clients, or equivalently, the number of orders, and $N$ denotes the number of drivers. Then, the final assignment will be recovered from the minimal value of column $C-1$ (numeration starts from 0) by backtracking.

## 3. Linear Programming

The third approach to the problem would be to use LP-relaxation. Given the variable matrix $A$ of size $N \times C$, where $C$ denotes the number of clients, and $N$ denotes the number of drivers, $a_{i,j} = 1$, if driver $i$ was assigned to order $j$, and $0$ otherwise.

Formulating the car allocation problem will require implementation of sophisticated restrictions and weights function. 

## 4. Integer Programming

## 5. Flow Problem

# Experiments

In [ ]:
# Miscellaneous here

# TODO: function to draw solutions to debug more easily

In [ ]:
# Greedy algorithm soluions here

In [59]:
# Dynamic programming solutions here

import numpy as np


## SIMPLE IMPLEMENTATION OF THE DYNAMIC PROGRAMMING:
## Assumptions:
## 1. Clients are sorted by the pickup desired time
## 2. Clients are picked up in the order of their pickup desired time


# Data Mock so far, not real data

# drivers amount
m = 4

# driver's initial positions
dr_x = np.asarray([10.0, 0.0, 20.0, 30.0])
dr_y = np.asarray([10.0, 0.0, 20.0, 30.0])

# clients amount
n = 3

# time desired
td = np.asarray([ 10, 10, 10])

# pickup point
xp = np.asarray([ 10.0, 20.0, 30.0])
yp = np.asarray([ 10.0, 20.0, 30.0])

# destination point
xd = np.asarray([ 10.0, 20.0, 30.0])
yd = np.asarray([ 20.0, 30.0, 40.0])


table = np.zeros((m, n + 1))

# positions for a driver at each point of the solution
d_x_positions = np.zeros((m, n + 1))
d_y_positions = np.zeros((m, n + 1))
d_x_positions[:, 0] = dr_x
d_y_positions[:, 0] = dr_y

d_times = np.zeros((m, n + 1))
assignments = np.zeros(n)

def distance(x1, y1, x2, y2):
    return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

for i_c in range(1, n + 1):  # iterating by clients
    d_x_positions[:, i_c] = d_x_positions[:, i_c - 1]
    d_y_positions[:, i_c] = d_y_positions[:, i_c - 1]
    d_times[:, i_c] = d_times[:, i_c - 1]

    # desired time of current client
    desired_time = td[i_c - 1]
    
    # point of pickup of current client
    pickup_x = xp[i_c - 1]
    pickup_y = yp[i_c - 1]
    
    # min driver
    i_min = 0
    min_penalty = np.inf
    for i_d in range(m):  # iterating by drivers
        distance_to_pickup = distance(d_x_positions[i_d, i_c - 1], d_y_positions[i_d, i_c - 1], pickup_x, pickup_y)
        lateness = np.maximum(0.0, d_times[i_d, i_c] + distance_to_pickup - desired_time) # travelling speed 1.0
                
        total_penalty = distance_to_pickup + lateness ** 4
                
        if min_penalty > total_penalty:
            min_penalty = total_penalty
            i_min = i_d
    
    # assignment index of current client
    assignments[i_c - 1] = i_min
    
    # updating the position of the assigned driver to the destination point of the client
    d_x_positions[i_min, i_c] = xd[i_c - 1]
    d_y_positions[i_min, i_c] = yd[i_c - 1]
    
    # updating time of the driver after delivering the client
    # it is max from desired time or time of pick up + time that is spent on delivering
    d_times[i_min, i_c] = \
                np.maximum(td[i_c - 1], d_times[i_min, i_c - 1] + distance(d_x_positions[i_min, i_c - 1], d_y_positions[i_min, i_c - 1], pickup_x, pickup_y)) + \
                                      distance(pickup_x, pickup_y, xd[i_c - 1], yd[i_c - 1])
            
print "Assignments of the drivers for clients", assignments

Assignments of the drivers for clients [ 0.  2.  3.]
